# Baseline

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from pathlib import Path

In [ ]:
data_path = Path("./data")
images_path = data_path / "competition/competition/"
masks_path  = data_path / "masks/"

In [ ]:
from utils import *

In [ ]:
ious = []

for index in range(5):
    name = f"{index:06d}.jpg"
    image_path = images_path / name
    mask_path  = masks_path  / name
    
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    true_mask = cv2.imread(str(mask_path))
    true_mask = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    true_mask = cv2.normalize(true_mask, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    _, true_mask = cv2.threshold(true_mask, 0.5, 1.0, cv2.THRESH_BINARY)
    
    mask = baseline_mask(image)
    
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.show()
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(mask)
    plt.show()
    fig = plt.figure(figsize=(10, 10))
    plt.imshow(true_mask)
    plt.show()
    plt.close()
    
    intersection_over_union = np.count_nonzero(np.minimum(mask, true_mask)) / np.count_nonzero(np.maximum(mask, true_mask))
    #print(intersection_over_union)
    ious.append(intersection_over_union)

In [ ]:
sum(ious) / 5

In [ ]:
for index in range(26600, 26725):
    print(index)
    name = f"{index:06d}.jpg"
    image_path = images_path / name
    
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    mask = baseline_mask(image)
    plot_with_mask(image, mask)
    
    plt.close()

In [ ]:
def df_from_mask(name, mask):
    numbers = pd.Series(np.arange(mask.size), name = 'ID')
    ind = numbers.apply(lambda n: name + f"_{n // mask.shape[1]}_{n % mask.shape[1]}")
    return pd.DataFrame({'value': mask.flatten()}, index = ind, dtype=np.int32)

In [ ]:
from tqdm import tqdm

dataframes = []

for index in tqdm(range(26600, 26725)):
    name = f"{index:06d}"
    image_path = images_path / (name + ".jpg")
    
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    
    mask = baseline_mask(image).astype(bool)
    
    #answer = pd.concat([answer, df_from_mask(name, mask)])
    dataframes.append(df_from_mask(name, mask))
    
    #fig = plt.figure(figsize=(10, 10))
    #plt.imshow(mask)
    #plt.show()

In [ ]:
answer = pd.concat(dataframes)
answer.to_csv('baseline.csv')